In [ ]:
#!{sys.executable} -m pip install --upgrade psycopg2-binary

In [1]:
# ket noi
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Kết nối với user postgres
conn = psycopg2.connect(
    host="localhost",
    user="postgres",       # user mặc định
    password="123",           # nếu có mật khẩu cho postgres thì điền vào
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()


In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Your provided connection code
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

try:
    # Set the isolation level for DDL commands
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

    # Get a list of all tables in the public schema
    print("--- Đang truy vấn các bảng và cột trong database video_db... ---\n")
    cursor.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
    """)
    tables = cursor.fetchall()

    if not tables:
        print("Không tìm thấy bảng nào trong database 'video_db'.")
    else:
        # Loop through each table to get its columns
        for table in tables:
            table_name = table[0]
            print(f"--- Bảng: {table_name} ---")

            # Get column details for the current table
            cursor.execute(f"""
                SELECT 
                    column_name, 
                    data_type, 
                    is_nullable 
                FROM information_schema.columns 
                WHERE table_name = '{table_name}' 
                ORDER BY ordinal_position
            """)
            columns = cursor.fetchall()

            # Print the columns neatly
            for col in columns:
                column_name, data_type, is_nullable = col
                null_status = "NULLABLE" if is_nullable == 'YES' else "NOT NULL"
                print(f"  -> Cột: {column_name:<25} | Kiểu dữ liệu: {data_type:<15} | Null: {null_status}")
            print("-" * 35)

except (Exception, psycopg2.Error) as error:
    print("Có lỗi xảy ra khi kết nối hoặc truy vấn:", error)

finally:
    # Close the cursor and connection
    if conn:
        cursor.close()
        conn.close()
        print("\nKết nối PostgreSQL đã đóng.")

--- Đang truy vấn các bảng và cột trong database video_db... ---

--- Bảng: frame_mappings ---
  -> Cột: frame_id                  | Kiểu dữ liệu: integer         | Null: NOT NULL
  -> Cột: video_id                  | Kiểu dữ liệu: integer         | Null: NULLABLE
  -> Cột: frame_path                | Kiểu dữ liệu: character varying | Null: NULLABLE
  -> Cột: pts_time                  | Kiểu dữ liệu: double precision | Null: NULLABLE
  -> Cột: frame_idx                 | Kiểu dữ liệu: integer         | Null: NULLABLE
  -> Cột: fps                       | Kiểu dữ liệu: integer         | Null: NULLABLE
  -> Cột: milvus_id                 | Kiểu dữ liệu: bigint          | Null: NULLABLE
-----------------------------------
--- Bảng: videos ---
  -> Cột: id                        | Kiểu dữ liệu: integer         | Null: NOT NULL
  -> Cột: video_path                | Kiểu dữ liệu: character varying | Null: NULLABLE
  -> Cột: title                     | Kiểu dữ liệu: character varying | Null: 

In [ ]:
# Tạo database nếu chưa có
# db_name = "video_db"
# cursor.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_name}';")
# exists = cursor.fetchone()
# if not exists:
#     cursor.execute(f"CREATE DATABASE {db_name};")
#     print(f"Database '{db_name}' created successfully!")
# else:
#     print(f"Database '{db_name}' already exists.")

# cursor.close()
# conn.close()

Database 'video_db' created successfully!


In [64]:
# Kết nối tới database vừa tạo
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"   # nếu user khác thì điền mật khẩu
)
cursor = conn.cursor()


In [36]:
create_tables_sql = """
CREATE TABLE IF NOT EXISTS videos (
    id SERIAL PRIMARY KEY,
    video_path VARCHAR(255) UNIQUE,
    title VARCHAR(255),
    description TEXT
);

CREATE TABLE IF NOT EXISTS frame_mappings (
    frame_id SERIAL PRIMARY KEY,
    video_id INTEGER REFERENCES videos(id),b
    frame_path VARCHAR(255) UNIQUE,
    pts_time FLOAT,
    frame_idx INT,
    fps INT,
    milvus_id BIGINT
);
"""

cursor.execute(create_tables_sql)
conn.commit()
print("Tables 'videos' and 'frame_mappings' created successfully!")


SyntaxError: syntax error at or near "VARCHAR"
LINE 12:     frame_path VARCHAR(255) UNIQUE,
                        ^


In [41]:
# check 
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = cursor.fetchall()
print("Tables in database:", tables)


Tables in database: [('frame_mappings',), ('videos',), ('media_info',)]


In [58]:
# Kiểm tra cột của bảng videos
cursor.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'videos';
""")
columns_videos = cursor.fetchall()
print("Columns in 'videos':")
for col in columns_videos:
    print(col)


Columns in 'videos':
('id', 'integer')
('video_path', 'character varying')
('title', 'character varying')
('description', 'text')


In [59]:
# Kiểm tra cột của bảng frame_mappings
cursor.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'frame_mappings';
""")
columns_frames = cursor.fetchall()
print("\nColumns in 'frame_mappings':")
for col in columns_frames:
    print(col)



Columns in 'frame_mappings':
('pts_time', 'double precision')
('video_id', 'integer')
('milvus_id', 'bigint')
('frame_id', 'integer')
('frame_idx', 'integer')
('fps', 'integer')
('frame_path', 'character varying')


In [68]:
# Kiểm tra cột của bảng media_info
cursor.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'media_info';
""")
columns_frames = cursor.fetchall()
print("\nColumns in 'media_info':")
for col in columns_frames:
    print(col)



Columns in 'media_info':
('id', 'integer')
('title', 'text')
('description', 'text')
('keywords', 'text')
('publish_date', 'text')
('length', 'integer')
('watch_url', 'text')


In [67]:
# Kiểm tra  của bảng videos ket hop voi media_info
cursor.execute("""
SELECT *
FROM media_info
""")
columns_frames = cursor.fetchall()
print("\nColumns in 'media_info':")
for col in columns_frames:
    print(col)



Columns in 'media_info':
(2257, 'BẮP CẢI HẤP NẤM TÔM  MÓN NGON MỖI NGÀY  VIVU TV', '"Món ngon mỗi ngày" phát sóng lúc 11g50 hàng ngày, trên HTV7\n“THẾ GIỚI ẨM THỰC - MÓN NGON MỖI NGÀY” thực đơn gia đình với những món ngon dễ làm lạ mắt và hấp dẫn. Mỗi ngày một món ngon cho bữa cơm gia đình ấm áp, mỗi điểm đến một món đặc sản hấp dẫn... Đón xem bộ sưu tập Thế giới ẩm thực đậm đà hương vị trên kênh Youtube VIVU TV\nNGUYÊN LIỆU   \n ********************\n#VIVUTV #monngonmoingay #BapCaiHapNamTom\nBạn giúp LIKE, SHARE và COMMENT nếu thấy hữu ích nhé !\n►Tổng hợp MÓN NGON MỖI NGÀY: https://bit.ly/2UEvaiP\n► Đăng ký: https://bit.ly/2WPXBNp\n► Facebook:www.facebook.com/life.htv/ \n► Tất cả video của ViVu TV Đều Có Bản Quyền\n► Cấm Sao Chép Dưới Mọi Hình Thức Contact information:\n* Your full legal name: ViVu TV', 'món ngon đơn giản món ngon mổi ngày VIVUTV món ngon món ngon dể làm salad gà nướng ẩm thực mon ngon moi ngay món ngon mỗi ngày MÓN NGON MỖI NGÀY VIVU TV bữa cơm gia đình món ăn việt

In [44]:
# Xóa toàn bộ dữ liệu nhưng giữ cấu trúc bảng
# cursor.execute("TRUNCATE TABLE frame_mappings RESTART IDENTITY CASCADE;")
cursor.execute("TRUNCATE TABLE media_info RESTART IDENTITY CASCADE;")
conn.commit()
print("✅ Dữ liệu trong 2 bảng 'media_info' và '' đã được xóa.")


✅ Dữ liệu trong 2 bảng 'media_info' và '' đã được xóa.


In [65]:
# kiem tra data
# ================= Kiểm tra bảng videos =================
cursor.execute("SELECT * FROM videos LIMIT 10;")
videos_rows = cursor.fetchall()
print("✅ 10 dòng đầu bảng videos:")
for row in videos_rows:
    print(row)

# ================= Kiểm tra bảng frame_mappings =================
cursor.execute("SELECT * FROM frame_mappings LIMIT 10;")
frame_rows = cursor.fetchall()
print("\n✅ 10 dòng đầu bảng frame_mappings:")
for row in frame_rows:
    print(row)

# ================= Kiểm tra bảng media_info=========
cursor.execute("SELECT * FROM media_info LIMIT 10;")
frame_rows = cursor.fetchall()
print("\n✅ 10 dòng đầu bảng media_info:")
for row in frame_rows:
    print(row)


✅ 10 dòng đầu bảng videos:
(1, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V002.mp4', 'L27_V002.mp4', 'Video demo')
(2, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V010.mp4', 'L27_V010.mp4', 'Video demo')
(3, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V007.mp4', 'L27_V007.mp4', 'Video demo')
(4, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V005.mp4', 'L27_V005.mp4', 'Video demo')
(5, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V006.mp4', 'L27_V006.mp4', 'Video demo')
(6, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V001.mp4', 'L27_V001.mp4', 'Video demo')
(7, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V015.mp4', 'L27_V015.mp4', 'Video demo')
(8, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V012.mp4', 'L27_V012.mp4', 'Video demo')
(9, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V004.mp4', 'L27_V004.mp4', 'Video demo')
(10, '/home/shared/data_batch1/Videos/videos_L27_a/L27_V008.mp4', 'L27_V008.mp4', 'Video demo')

✅ 10 dòng đầu bảng fr

# Bước 1: Thêm cột vào moii media_info ( luu thon tin file json)

In [ ]:
import psycopg2

# Kết nối database
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

# Tạo bảng video_metadata
cursor.execute("""
    CREATE TABLE IF NOT EXISTS media_info (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        keywords TEXT,
        publish_date TEXT,
        length INTEGER,
        watch_url TEXT,
        FOREIGN KEY (id) REFERENCES videos(id) ON DELETE CASCADE
    );
""")
conn.commit()
print("Đã tạo bảng media_info thành công.")

# Kiểm tra cấu trúc bảng
cursor.execute("""
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns
    WHERE table_name = 'media_info';
""")
columns = cursor.fetchall()
print("Columns in 'media_info':")
for col in columns:
    print(col)

# Đóng kết nối
cursor.close()
conn.close()

Đã tạo bảng media_info thành công.
Columns in 'media_info':
('length', 'integer', 'YES')
('id', 'integer', 'NO')
('description', 'text', 'YES')
('publish_date', 'text', 'YES')
('watch_url', 'text', 'YES')
('keywords', 'text', 'YES')
('title', 'text', 'NO')


# Bước 2: Code xử lý file JSON và insert vào video_db           

In [46]:
import psycopg2

# Kết nối database
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

# Đếm số bản ghi trong videos
cursor.execute("SELECT COUNT(*) FROM videos;")
print(f"Số bản ghi trong bảng videos: {cursor.fetchone()[0]}")

# Lấy danh sách video_path
cursor.execute("SELECT video_path FROM videos")
print("Danh sách video_path mẫu:")
for row in cursor.fetchall():
    print(row[0])

# Đóng kết nối
cursor.close()
conn.close()

Số bản ghi trong bảng videos: 873
Danh sách video_path mẫu:
/home/shared/data_batch1/Videos/videos_L27_a/L27_V002.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V010.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V007.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V005.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V006.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V001.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V015.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V012.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V004.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V008.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V014.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V013.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V011.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V016.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V003.mp4
/home/shared/data_batch1/Videos/videos_L27_a/L27_V009.mp4
/home/shared

In [47]:
# xử lý JSON và insert vào bảng media_info
import os
import json
import pandas as pd
import psycopg2
import time

# Config
JSON_DIR = "/home/shared/data_batch1/media-info/"
VIDEO_BASE_DIR = "/home/shared/data_batch1/Videos/"

# Kết nối database
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

# Hàm load tất cả file JSON
def load_json_metadata():
    json_data = []
    json_count = 0
    start_time = time.time()
    for filename in os.listdir(JSON_DIR):
        if filename.endswith('.json'):
            filepath = os.path.join(JSON_DIR, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    video_id = filename.replace('.json', '')  # Ví dụ: L21_V001
                    json_data.append({
                        'video_id': video_id,
                        'title': data.get('title', ''),
                        'description': data.get('description', ''),
                        'keywords': ' '.join(data.get('keywords', [])),
                        'publish_date': data.get('publish_date', ''),
                        'length': data.get('length', 0),
                        'watch_url': data.get('watch_url', '')
                    })
                    json_count += 1
                except json.JSONDecodeError:
                    print(f"Lỗi load file: {filename}")
    end_time = time.time()
    print(f"Đã load {json_count} file JSON trong {end_time - start_time:.2f} giây.")
    return pd.DataFrame(json_data)

# Hàm kiểm tra file video tồn tại
def find_video_path(video_id):
    group = video_id.split('_')[0]  # Ví dụ: L21 từ L21_V001
    for subgroup in os.listdir(VIDEO_BASE_DIR):
        video_path = os.path.join(VIDEO_BASE_DIR, subgroup, f"{video_id}.mp4")
        if os.path.exists(video_path):
            return video_path
    return None

# Hàm insert hoặc update metadata vào bảng media_info
def insert_json_to_media_info(df):
    batch_size = 100  # Số lượng bản ghi xử lý mỗi lần
    start_time = time.time()
    for i in range(0, len(df), batch_size):
        batch = df[i:i + batch_size]
        values = []
        for _, row in batch.iterrows():
            video_path = find_video_path(row['video_id'])
            if video_path:
                # Lấy id từ bảng videos dựa trên video_path
                cursor.execute("SELECT id FROM videos WHERE video_path = %s;", (video_path,))
                video_id = cursor.fetchone()
                if video_id:
                    values.append((
                        video_id[0],  # id từ bảng videos
                        row['title'],
                        row['description'],
                        row['keywords'],
                        row['publish_date'],
                        row['length'],
                        row['watch_url']
                    ))
                else:
                    print(f"Không tìm thấy id cho video_path {video_path} trong bảng videos.")
            else:
                print(f"Không tìm thấy file video cho {row['video_id']}.")

        try:
            if values:  # Chỉ insert nếu có dữ liệu
                cursor.executemany("""
                    INSERT INTO media_info (id, title, description, keywords, publish_date, length, watch_url)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (id) DO UPDATE
                    SET title=EXCLUDED.title,
                        description=EXCLUDED.description,
                        keywords=EXCLUDED.keywords,
                        publish_date=EXCLUDED.publish_date,
                        length=EXCLUDED.length,
                        watch_url=EXCLUDED.watch_url
                """, values)
                conn.commit()
        except Exception as e:
            print(f"Lỗi xử lý batch {i//batch_size}: {e}")
            conn.rollback()
    end_time = time.time()
    print(f"Đã cập nhật/insert {len(df)} metadata vào bảng media_info trong {end_time - start_time:.2f} giây.")

# Chạy pipeline
try:
    start_time = time.time()
    json_df = load_json_metadata()
    insert_json_to_media_info(json_df)
    end_time = time.time()
    print(f"Thời gian tổng cộng: {end_time - start_time:.2f} giây.")
except Exception as e:
    print(f"Lỗi trong pipeline: {e}")
finally:
    cursor.close()
    conn.close()

Đã load 873 file JSON trong 0.03 giây.
Đã cập nhật/insert 873 metadata vào bảng media_info trong 0.37 giây.
Thời gian tổng cộng: 0.40 giây.


In [50]:
# Kết nối database
conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

In [51]:
cursor.execute("SELECT COUNT(*) FROM media_info;")
print(f"Số bản ghi trong bảng media_info: {cursor.fetchone()[0]}")

Số bản ghi trong bảng media_info: 873


In [52]:
cursor.execute("SELECT id, title, keywords, publish_date FROM media_info LIMIT 5;")
print(cursor.fetchall())

[(2257, 'BẮP CẢI HẤP NẤM TÔM  MÓN NGON MỖI NGÀY  VIVU TV', 'món ngon đơn giản món ngon mổi ngày VIVUTV món ngon món ngon dể làm salad gà nướng ẩm thực mon ngon moi ngay món ngon mỗi ngày MÓN NGON MỖI NGÀY VIVU TV bữa cơm gia đình món ăn việt nam thực đơn mổi ngày thực đơn món ăn ngon VIVU TV mon an ngon moi ngay thức ăn cơm gia dinh món ăn ngon mổi ngày mnmn MNMN gan bo ap chao chế biến lòng bò ngon gỏi hoa chuối thịt gà chay bắp cải hấp nấm tôm', '27/01/2022'), (2061, 'CÁ KHO QUẸT MÓN QUÊ LẠ MIỆNG  MÓN NGON MỖI NGÀY  VIVU TV 16052020.', 'VIVUTV VIVU TV món ngon ẩm thực thực đơn thực đơn mổi ngày gia đình cơm gia đình cơm gia dinh món ngon mỗi ngày mon ngon moi ngay nấu nướng bữa cơm món ngon dể làm thức ăn đảm đang món ngon đơn giản học nấu ăn dạy nấu ăn món ăn dể làm nội trợ bếp nuc vợ đảm MÓN NGON MỖI NGÀY VIVU TV món ăn việt nam nấu ăn ngon dau hu cuon chien gion cá kho quẹt kho quẹt món quê lạ miệng', '17/05/2020'), (1977, 'CƠM TÔM GIÒN KHO QUẸT  MÓN NGON MỖI NGÀY  VIVU TV', 'VIVU

In [53]:
cursor.execute("SELECT COUNT(*) FROM media_info WHERE id NOT IN (SELECT id FROM videos);")
print(f"Số bản ghi trong media_info không khớp với videos: {cursor.fetchone()[0]}")

Số bản ghi trong media_info không khớp với videos: 0


In [54]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="video_db",
    user="postgres",
    password="123"
)
cursor = conn.cursor()

cursor.execute("SELECT id, title, keywords, publish_date FROM media_info LIMIT 5;")
print(cursor.fetchall())

cursor.close()
conn.close()

[(2257, 'BẮP CẢI HẤP NẤM TÔM  MÓN NGON MỖI NGÀY  VIVU TV', 'món ngon đơn giản món ngon mổi ngày VIVUTV món ngon món ngon dể làm salad gà nướng ẩm thực mon ngon moi ngay món ngon mỗi ngày MÓN NGON MỖI NGÀY VIVU TV bữa cơm gia đình món ăn việt nam thực đơn mổi ngày thực đơn món ăn ngon VIVU TV mon an ngon moi ngay thức ăn cơm gia dinh món ăn ngon mổi ngày mnmn MNMN gan bo ap chao chế biến lòng bò ngon gỏi hoa chuối thịt gà chay bắp cải hấp nấm tôm', '27/01/2022'), (2061, 'CÁ KHO QUẸT MÓN QUÊ LẠ MIỆNG  MÓN NGON MỖI NGÀY  VIVU TV 16052020.', 'VIVUTV VIVU TV món ngon ẩm thực thực đơn thực đơn mổi ngày gia đình cơm gia đình cơm gia dinh món ngon mỗi ngày mon ngon moi ngay nấu nướng bữa cơm món ngon dể làm thức ăn đảm đang món ngon đơn giản học nấu ăn dạy nấu ăn món ăn dể làm nội trợ bếp nuc vợ đảm MÓN NGON MỖI NGÀY VIVU TV món ăn việt nam nấu ăn ngon dau hu cuon chien gion cá kho quẹt kho quẹt món quê lạ miệng', '17/05/2020'), (1977, 'CƠM TÔM GIÒN KHO QUẸT  MÓN NGON MỖI NGÀY  VIVU TV', 'VIVU